# Test ComCam photodiode

Craig Lage - 28-Jul-24

In [ ]:
import sys, asyncio, time, os
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.io import fits

from lsst.ts import salobj

## Now connect to ComCam and the Electrometer CSC

In [ ]:
# change this to a local directory
data_dir = '/home/cslage/DATA'

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
domain = salobj.Domain()
electrometer = salobj.Remote(name="Electrometer", domain=domain, index=101) # Check SAL index

In [ ]:
await electrometer.start_task

In [ ]:
await electrometer.cmd_setLogLevel.set_start(level=10)

In [ ]:
elec_tmp = await electrometer.evt_summaryState.aget()
print('Electrometer: ',salobj.State(elec_tmp.summaryState))

In [ ]:
elec_tmp.salIndex

In [ ]:
test = electrometer.evt_detailedState.get()
print(test)

# Test the Electrometer.

In [ ]:
await electrometer.cmd_setRange.set_start(setRange=2E-8) # -1 is autorange

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

In [ ]:
exp_time = 1 #sec
electrometer.evt_largeFileObjectAvailable.flush()
await electrometer.cmd_startScan.set_start(timeout=10)
await asyncio.sleep(exp_time)
await electrometer.cmd_stopScan.set_start(timeout=exp_time+20)

In [ ]:
lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename= os.path.split(lfa.url)[1]
print(filename)
elec_filen = os.path.join(data_dir, filename)
os.system(f"curl {lfa.url} --output {elec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
data = hdu[1].data
print(data)

In [ ]:
electrometer_mode = 1
range = 2E-8
integration_time = 0.1

await electrometer.cmd_setMode.set_start(
            mode=electrometer_mode,
        )
await electrometer.cmd_setRange.set_start(
    setRange=range,
)
await electrometer.cmd_setIntegrationTime.set_start(
    intTime=integration_time,
)
await electrometer.cmd_performZeroCalib.start()
await electrometer.cmd_setDigitalFilter.set_start(
    activateFilter=False,
    activateAvgFilter=False,
    activateMedFilter=False,
)

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

In [ ]:
exp_time = 1 #sec
electrometer.evt_largeFileObjectAvailable.flush()
await electrometer.cmd_startScan.set_start(timeout=10)
await asyncio.sleep(exp_time)
await electrometer.cmd_stopScan.set_start(timeout=exp_time+20)

In [ ]:
lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename= os.path.split(lfa.url)[1]
print(filename)
elec_filen = os.path.join(data_dir, filename)
os.system(f"curl {lfa.url} --output {elec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
data = hdu[1].data
print(data)

In [ ]:
url='https://s3.cp.lsst.org/rubinobs-lfa-cp/Electrometer:101/fits/2024/07/29/Electrometer:101_fits_2024-07-29T21:09:15.986.fits'
filename= os.path.split(url)[1]
print(filename)
elec_filen = os.path.join(data_dir, filename)
os.system(f"curl {url} --output {elec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
data = hdu[1].data
print(data)

In [ ]:
vals = []
for dat in data:
    vals.append(dat[0])
print(np.mean(vals), np.std(vals))

In [ ]:
# Plot Data
timestamp = filename.split('_')[2].split('.')[0]
plt.plot(data['Elapsed Time'], data['Signal'],marker='x')
plt.xlabel('ElapsedTime')
plt.ylabel('Signal')
plt.ylim(-2E-8, 0)
plt.savefig(f"/home/cslage/DATA/ComCam_Keysight_Scan_{timestamp}.png")

In [ ]:
hdu[0].header